## 1.0 Problem Formulation and Data Acquisition ##

In this notebook, I'll walk-through an explanation of my goals and cover the foundational work needed to reach those goals;  extraction, cleaning and manipulation culminating into a desired representation of the data.

### 1.1 Introduction to the Problem/ Fantasy Football Primer
<br>


<font size=4>__Key Takeaways__</font>

 - Fantasy Football offers a lot of interesting data science tasks/problems to explore. One of those is predicting players a fantasy owner should choose.
 - Choosing the right players is crucial for being competitive in a league.
 - Aside from choosing a player based on performance (which has a more direct relationship with points, modeling  potential for injury by using past player injured reserve (ir) data could inform owners about players to avoid
 - One approach is to represent a players' career as a vector with each index in the vector representing a single season and each value representing whether the player was healthy or an injury that caused the player to be placed on IR.


<font size=4>__Problem Formulation__</font>
<br>
<font size=10>F</font>
<font size=4>or this analysis, I'll be focusing on the "Who should I chose?" question.  My long-term desire is to build a reasonably accurate predictive model to aide in choosing a player for an upcoming season. That simple question becomes very complex when considering the variables needed to predict a player's future success.  You may think that a prior season's performance is a good predictor of the future but we could reason that modeling future performance based on prior statistics is most likely a simplistic representation of the complex real-world. But what other factors influence a player's performance?  Here, I would make a distinction between intrinsic (player-dependent) and extrinsic (outside of the player) factors. Beyond past performance, I would suggest that intrinsic factors could include age, experience, or even potentially height/weight as examples.  Extrinsic features could include a player's coach, the talent surrounding them, even the franchise they belong to.  We can't faithfully model all of the real-world factors, but we can try our best to understand what factors have the greatest influence on a player's performance and model these.  This concept is called large-world uncertainty.

<font size=4>
In my question of "Whom should I choose?" and brainstorming the intrinic/extrinsic factors I found a bias in my own thinking. I was seeking out the obvious - variables that discretely generate points (touchdowns, rushing yards, catches, etc. and are positively correlated to fantasy points). I became a little more sophisticated and reasoned that age might play a role as it's been suggested and mostly observed that as a player ages, their performance is likely to diminish; a widely observed relationship in sports is that age has a negative correlation to production.  Then the most obvious thing finally dawned on me - what about injuries?  If a player is injured, they can't play.  If they can't play, they can't generate points and that could have just as much of an impact on a fantasy team than using some other variable. So, if I choose a high performing player (we'll call them "A")  and they get injured and are out several games, than their higher performances are diluted over the span of the season as they aren't generating points during the games they're injured (ie their mean-season performance is decreased).  Now, the prospect of grabbing a player who is a moderate performer ("B") but stays healthy across the season might not seem like such a bad choice.  I'll point out that this is not black and white scenario - it might just be that A won your team games for a few weeks whereas playing B in those same weeks wouldn't have led to the same result so it's possible the risk of selecting A if there was a prior belief that they would get injured might still be worth it, but in general, especially during a draft, you don't have any certainty about future performance - and therefore, your limited to predicting the future.


<font size=4><p>  
It's reasonable to assume, however, that an owner would want to avoid prolonged injuries as a risk-mitigations strategy for earlier selection where the very best players are presumed to still be available. Considering other owners will be selecting the highest expected performers early in the draft, owners choosing players who are high performers and less at risk of prolonged injury ensures that they'll be able to stay competitive. Therefore, in asking the question, "Whom should I choose?", it's important to ask an intersecting question of "Whom is likely to be injured?".  "Injured" or more generally "Health" can be multi-faceted (players could have minor injuries that take them out of a game, long-term injuries that require them to be sit out a predefined number of weeks with a designation of "Injured Reserve", or they could catch covid and sit out of games until they recover).  

<font size=4>
In this analysis, I explore this question using the presense/non-presence of a player's stint on Injured Reserve as a proxy for health, as long-term inactivity (from injury) will have a significant impact on fantasy performance.</font><p>

    

<font size=4> __Data Representation__ </font><p>
For this first attempt, We're going to build a simple model using vector representations of player careers. Our resulting model will be fairly simplistic as it will only be looking at the probability of injury given a sequence of prior injuries (using markov chains).  Real-life is obviously much more nuanced but I love thinking about data in terms of the toolsets I've learned from my courses and experimenting with their application. We'll explore alternative representations and techniques in the future.
    
 
With that, let's get our data for our analysis...

In [125]:
# Let's import the important libraries for our work.

import pandas as pd
import numpy as np
import math
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
from sqlalchemy import create_engine
import ast

### 1.1 Data Scraping 

Our first step is to get a list of players that we can perform our analysis on.  The best way to do this is to cycle through one or more years of NFL Drafts and retrieve each drafted player and their player page.  This won't give us a complete list of players in the NFL but it should give us a good amount of data to work with. Here's an example of a page we'll be scraping: https://www.footballdb.com/draft/draft.html?yr=2022

In [59]:
def getSoup(url,head):
    resp = requests.get(url,headers=head)
    soup = BeautifulSoup(resp.content)
    return soup

In [29]:
def GetDraftees(draft_years):
    
    """
    Produces a dictionary of players selected during an NFL Draft based on a provided list of seasons
    
    Parameters:
        draft_years (list): a list of 4-digit years corresponding to NFL Drafts of interest
    Returns:
        dictionary of each desired year's NFL Draft picks
    """
    
    
    head = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15"}
    year_lst=[]
    dround=[]
    pick=[]
    team = []
    player = []
    ppage = []
    position = []
    school = []

   
    for year in tqdm(draft_years):
        for rnd in range(1,8):
            time.sleep(2)
            url = f"https://www.footballdb.com/draft/draft.html?lg=NFL&yr={year}&rnd={rnd}"
            resp = requests.get(url,headers=head)
            soup = BeautifulSoup(resp.content)
            table = soup.find_all('div', class_='tr')
            for row in tqdm(table[1:]):        
                cell_lst = row.findChildren('div')
                # try to retrieve player's page so we can scrap transactions
                try:
                    page = cell_lst[2].findChildren('a',href=True)[1]['href']
                # account for instances in which we can't get a page
                except:
                    page = "unavailable"
                    seasons = []
                position.append(cell_lst[4].get_text())
                school.append(cell_lst[5].get_text())
                # keep a small memory footprint by yielding out the results as a generator
                if page:
                    # Get the seasons that the player played from their career stats
                    url = f"https://www.footballdb.com/{page}"
                    soup = getSoup(url,head)
                    resp = requests.get(url,headers=head)
                    soup = BeautifulSoup(resp.content)
                    rows = soup.find_all('tbody')[0].find_all('tr')
                    seasons = [td[0].get_text() for td in [row.find_all('td') for row in rows] if td[2].get_text() == 'NFL' and len(td[0].get_text()) > 3]
                
                
                yield {'year': year,
                       'round': cell_lst[0].get_text(),
                       'pick' : cell_lst[1].get_text(),
                       'team' : cell_lst[2].findChildren('b')[0].get_text(),
                       'player': cell_lst[3].get_text(),
                       'page' : page,
                       'position': cell_lst[4].get_text(),
                       'season': seasons
                      }



In [30]:
# 2013 was the first year that IR data appears to be collected for this website
lst  = [x for x in range(2013,2024)]
df = pd.DataFrame(data=GetDraftees(lst))
df.head()


100%|██████████| 32/32 [00:42<00:00,  1.33s/it]

100%|██████████| 30/30 [00:43<00:00,  1.46s/it]

100%|██████████| 35/35 [00:51<00:00,  1.47s/it]

100%|██████████| 36/36 [00:46<00:00,  1.28s/it]

100%|██████████| 35/35 [00:46<00:00,  1.33s/it]

100%|██████████| 38/38 [00:50<00:00,  1.32s/it]

100%|██████████| 32/32 [00:52<00:00,  1.64s/it]

100%|██████████| 32/32 [00:46<00:00,  1.46s/it]

100%|██████████| 36/36 [00:48<00:00,  1.35s/it]

100%|██████████| 40/40 [00:56<00:00,  1.40s/it]

100%|██████████| 36/36 [00:47<00:00,  1.32s/it]

100%|██████████| 39/39 [00:45<00:00,  1.17s/it]

100%|██████████| 32/32 [00:45<00:00,  1.43s/it]

100%|██████████| 32/32 [00:39<00:00,  1.22s/it]

100%|██████████| 35/35 [00:44<00:00,  1.27s/it]

100%|██████████| 37/37 [00:52<00:00,  1.42s/it]

100%|██████████| 40/40 [00:56<00:00,  1.41s/it]

100%|██████████| 41/41 [00:57<00:00,  1.39s/it]

100%|██████████| 31/31 [00:43<00:00,  1.41s/it]

100%|██████████| 32/32 [00:43<00:00,  1.35s/it]

100%|██████████| 35/

,year,round,pick,team,player,page,position,season
0,2013,1,1,Kansas City Chiefs,Eric Fisher,/players/eric-fisher-fisheer01,OT,"[2013, 2014, 2015, 2016, 2017, 2018, 2019, 202..."
1,2013,1,2,Jacksonville Jaguars,Luke Joeckel,/players/luke-joeckel-joecklu01,OT,"[2013, 2014, 2015, 2016, 2017]"
2,2013,1,3,Miami Dolphins,Dion Jordan,/players/dion-jordan-jordadi01,DE,"[2013, 2014, 2017, 2018, 2019, 2020]"
3,2013,1,4,Philadelphia Eagles,Lane Johnson,/players/lane-johnson-johnsla06,OT,"[2013, 2014, 2015, 2016, 2017, 2018, 2019, 202..."
4,2013,1,5,Detroit Lions,Ezekiel Ansah,/players/ezekiel-ansah-ansahez01,DE,"[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]"


In [31]:
df.head()

,year,round,pick,team,player,page,position,season
0,2013,1,1,Kansas City Chiefs,Eric Fisher,/players/eric-fisher-fisheer01,OT,"[2013, 2014, 2015, 2016, 2017, 2018, 2019, 202..."
1,2013,1,2,Jacksonville Jaguars,Luke Joeckel,/players/luke-joeckel-joecklu01,OT,"[2013, 2014, 2015, 2016, 2017]"
2,2013,1,3,Miami Dolphins,Dion Jordan,/players/dion-jordan-jordadi01,DE,"[2013, 2014, 2017, 2018, 2019, 2020]"
3,2013,1,4,Philadelphia Eagles,Lane Johnson,/players/lane-johnson-johnsla06,OT,"[2013, 2014, 2015, 2016, 2017, 2018, 2019, 202..."
4,2013,1,5,Detroit Lions,Ezekiel Ansah,/players/ezekiel-ansah-ansahez01,DE,"[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]"


Great! We have all of the NFL Draftees from 2014 to 2021 (2022 draftees haven't provided us much info at the time of this writing).  To prepare for the joining of the transaction data, I need to create rows  for every player between the time they were drafted and now.  Using only the transaction data will only provide us the seasons in which an IR transaction occurred, but we can't rely on it to account for every season. This is crucial, as we want to represent every season of a player's career (whether there was an injury or not). 

Let me illustrate this point by taking Derrick Henry's career represented as a sequence of his career with/without IR. With only transaction data, we get the following sequence: 

Derrick Henry:  [Foot] 

where <code>Foot</code> was an injury that occurred in 2021.  But Henry didn't play just one season.  So we need to represent his career as several <code>Healthy</code> seasons followed by an IR Injury.  Since he started his career in 2016, we'd want his career vector to look like this:

<code>['Healthy','Healthy','Healthy','Healthy','Healthy',Foot','Healthy','Healthy'] </code>

which would correspond (positionally to the following years:

<code>[2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]</code>


The following code prepares each player to be represented this way (prior to joining the IR data).  Note: Only run this cell once or run the prior cell first before the following cell to prevent unncessary duplication of rows.
    
   

In [12]:
# Create a list of seasons
lst_seasons = [x for x in range(2014,2024)]


# Create a pandas series that cycles through the number of players (represented as the # of pandas rows * the list of seasons we care about
seasons = pd.Series(df.shape[0] *  lst_seasons)


# Duplicate each player row by the length of the number of seasons (length of the list of seasons above).  
# Reset the index so that the series can match up to the df's index
idx = repeat(len(lst_seasons)
df = df.iloc[df.index.repeat(len(lst_seasons)),:].reset_index()


# Create a new column from the pandas series
df['season'] = seasons


# Let's remove any seasons that occurred prior to a player's draft year - because players don't play prior to being drafted

# df = df[(df['position'].isin(['QB','RB','WR','TE'])) &
#        (df['year'] <= df['season'])]

df = df[df['year'] <= df['season']]



df.head(15)

IndexError: positional indexers are out-of-bounds

In [38]:
# Save the intermediate file
df.to_csv("data/raw_data/2013_2024_Draft.csv",index=True)

In [161]:
#read in the file and read in the column of lists as approriate
df = pd.read_csv("data/raw_data/2013_2024_Draft.csv",index_col=0)
df['season'] = df['season'].apply(lambda x: ast.literal_eval(x))
df = df[df['year'] < 2024]

In [162]:
# Create a new row for each season that was captured in the player's stats page
df = df.explode('season')

In [163]:
# If we have a blank (nan) list, fill it in with the draft year
df['season'] = df.apply(lambda x: x['year'] if pd.isnull(x['season']) else x['season'],axis=1)

In [164]:
df['season'] = df['season'].astype('int64')

Ok, we have an accounting of the seasons and we've eliminated seasons that existed prior to the player's draft year, let's focus on the transaction data now...

## 1.2.0 Generate Transaction Data ##

Let's develop a generator function that scrapes all of the pages of the players who we obtained from every year's draft (Offensive positions only).  This will yield a data object containing each player's transaction history over their career.

In [25]:
head = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15"}
transdate = []
transteam = []
transaction = []
players = []
seasons = []



def GetTransactions(pages):
    
    """
    Purpose: This generator function retrieves transaction details for each player we extracted from the drafts.
    Parameters:
        pages - a list of footballdb.com partial urls of players we want to extract details from. This partial url is after
        the "https://www.footballdb.com/" base url
    Output:
        a python dictionary to be passed to a pandas DataFrame. This is a generator function so the memory footprint 
        is relatively manageable given the number of pages we're extracting from
    """"
    
    
    head = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15"}
    time.sleep(2)
    # player = r['player']
    # page = r['page']
    
    
    for page in tqdm(pages[:]):
        url = f"https://www.footballdb.com/{page}/transactions"
        soup = getSoup(url,head) #calling our function from above
        body = soup.find_all('tbody')[0]
        tr_lst = body.find_all('tr')
        for row in tr_lst:
            obj = {} # we create python dictionary to add the transactions details 
            cells = row.find_all('td')
            obj['page'] = page
            obj['transdate'] = cells[0].find_all('span')[0].get_text()
            obj['team'] = cells[1].find_all('span')[0].get_text()
            obj['transaction'] = cells[2].get_text()
            yield obj

        
        
pages = df.page.unique()      
trans = pd.DataFrame(GetTransactions(pages))      

 18%|█▊        | 501/2817 [15:02<1:09:32,  1.80s/it]


KeyboardInterrupt: 

In [7]:
pages = df.page.unique()   

In [166]:
# Check the new dataframe
trans.head()

,page,transdate,team,transaction,trans_year,trans_month
2297,/players/ryan-jensen-jensery01,2013-05-03,Baltimore (NFL),Signed,2013,5
2727,/players/aaron-mellette-melleaa01,2013-05-03,Baltimore (NFL),Signed,2013,5
1438,/players/kyle-juszczyk-juszcky01,2013-05-03,Baltimore (NFL),Signed,2013,5
2258,/players/kapron-lewismoore-lewiska01,2013-05-03,Baltimore (NFL),Signed,2013,5
2853,/players/marc-anthony-anthoma01,2013-05-03,Baltimore (NFL),Signed,2013,5


In [110]:
# Save the data so we don't have to scrape all over again
trans.to_csv('data/all_pos_full_transactions.csv', index=False)


In [349]:
# Let's read the transaction data into the csv
# take Derrick Henry and see what transactions he's generated (and whether we've done a good job collecting them)

trans = pd.read_csv('data/all_pos_full_transactions.csv')
trans[trans['page'].str.contains('derrick-henry.*')==True]


,page,transdate,team,transaction,trans_year,trans_month
2581,/players/derrick-henry-henryde01,2016-05-09,Tennessee (NFL),Signed,2016,5
10262,/players/derrick-henry-henryde01,2020-03-16,Tennessee (NFL),Designated as franchise player,2020,3
10453,/players/derrick-henry-henryde01,2020-04-02,Tennessee (NFL),Signed,2020,4
16807,/players/derrick-henry-henryde01,2021-11-01,Tennessee (NFL),Placed on Injured Reserve (Foot),2021,11
18975,/players/derrick-henry-henryde01,2022-01-05,Tennessee (NFL),Designated for return from Injured Reserve,2022,1
19332,/players/derrick-henry-henryde01,2022-01-21,Tennessee (NFL),Activated from Injured Reserve,2022,1
27524,/players/derrick-henry-henryde01,2024-03-14,Baltimore (NFL),Signed,2024,3


Perfect!  looks like we've captured all of the transactions when compared to his actual history on footballdb.com (as of this writing). We'll need to do some housecleaning including extracting the year/month out of the transactions and determining what season number the player was in for the particular ir transaction.  

One thing we need to be mindful of is that the transaction data is associated to the calendar date NOT the season year.  If a player is injured in January 2024, the attributable season would be 2023 since the NFL Season doesn't end until after the Super Bowl is completed (played in February). Prior to the join, we need to make a common column for both dataframes to reference the same season. The transaction dataframe needs to account for the season starting in month 9 (Sept) and carrying all the way to month 2(Feb).  

In [350]:
trans['transdate'] = pd.to_datetime(trans['transdate'])
trans['trans_year'] = trans['transdate'].dt.year
trans['trans_month'] = trans['transdate'].dt.month
trans['season'] = trans.apply(lambda x: x['trans_year'] - 1 \
                                if x['trans_month'] <=2 \
                                else x['trans_year'], 
                                axis=1)


Also we want to see if there are players who retired (referenced as specific transactions) which can help define when their career completed.

In [351]:
# Let's take a look to see what types of transactions include some mention of retire

print(trans.transaction.unique().reshape(-1,1))
print('Retirement related transactions: ', [x for x in list(trans.transaction.unique()) if 'retire' in x.lower()] )

[['Signed']
 ['Placed on the Reserve/Retired List']
 ['Placed on Injured Reserve (Knee)']
 ['Waived']
 ['Placed on Injured Reserve (Back)']
 ['Placed on Injured Reserve (Lisfranc)']
 ['Placed on Injured Reserve (Pectoral)']
 ['Placed on the Non-Football Injury List (Foot)']
 ['Placed on Injured Reserve (Shoulder)']
 ['Waived (injured)']
 ['Placed on the Physically Unable to Perform List (Knee)']
 ['Placed on the Physically Unable to Perform List (Foot)']
 ['Placed on the Non-Football Injury List (Knee)']
 ['Placed on Injured Reserve (Ankle)']
 ['Placed on the Physically Unable to Perform List (Ankle)']
 ['Placed on Injured Reserve (Toe)']
 ['Placed on Injured Reserve (Leg)']
 ['Placed on Injured Reserve (Hamstring)']
 ['Placed on Injured Reserve (Hand)']
 ['Signed to the Practice Squad']
 ['Acquired via waivers (from the Oakland Raiders)']
 ['Acquired via waivers (from the San Francisco 49ers)']
 ['Acquired via waivers (from the Houston Texans)']
 ['Placed on Injured Reserve (Designate

In [352]:
len(trans[trans['transaction']== 'Placed on the Reserve/Retired List'])

89

Hmm...only 89 cases where a player actually retired...looking through the list and comparing some players it seems like we would want to also check for cases where a player was released and was not picked up by another team.

In [353]:
trans = trans.sort_values('transdate',ascending=True)
last = trans.groupby('page')[['transdate','transaction']].agg({'transdate':'max','transaction':'last'}).reset_index()
last['end'] = last.apply(lambda x: 1 if  "Waived" in x['transaction'] else 0, axis=1)
last

,page,transdate,transaction,end
0,/players/aaron-banks-banksaa01,2021-05-13,Signed,0
1,/players/aaron-burbridge-burbraa01,2019-07-17,Waived,1
2,/players/aaron-colvin-colviaa01,2020-09-06,Signed to the Practice Squad,0
3,/players/aaron-dobson-dobsoaa01,2017-09-06,Released from Injured Reserve,0
4,/players/aaron-donald-donalaa01,2018-09-08,Reinstated,0
...,...,...,...,...
2812,/players/zay-jones-jonesis02,2024-05-13,Signed,0
2813,/players/zech-mcphearson-mcpheze01,2023-08-20,Placed on Injured Reserve (Achilles),0
2814,/players/zeke-motta-mottaze01,2015-04-02,Waived,1
2815,/players/zion-johnson-johnszi01,2022-05-31,Signed,0


In [354]:
# Check if we have any players with no seasons. In these cases, the player never got play time for
# a variety of reasons. We'll fill the season in with the same year as the draft so that things go smoothly

df['season'] = df.apply(lambda x: x['year'] if math.isnan(x['season']) else x['season'],axis =1)

In [355]:
# Get only the transactions we care about
ir = trans[(trans['transaction'].str.contains('Placed on Injured Reserve.*')==True)| 
           (trans['transaction'].str.contains('Placed on the Physically Unable to Perform List.*')==True) ]


# Make sure we only pull transactions that occurred during the player's NFL Career
ir = ir[ir['team'].str.contains('NFL')==True]


p_ir = df.merge(ir, how='left',
                left_on=['page','season'],
               right_on=['page','season'])

p_ir.rename({'index':'player_id'},axis=1,inplace=True)
print(len(p_ir))


12039


In [356]:
p_ir[p_ir['player']=='Derrick Henry']

,year,round,pick,draft_team,player,page,position,season,transdate,team,transaction,trans_year,trans_month
4762,2016,2,45,Tennessee Titans,Derrick Henry,/players/derrick-henry-henryde01,RB,2016,NaT,NaN,NaN,NaN,NaN
4763,2016,2,45,Tennessee Titans,Derrick Henry,/players/derrick-henry-henryde01,RB,2017,NaT,NaN,NaN,NaN,NaN
4764,2016,2,45,Tennessee Titans,Derrick Henry,/players/derrick-henry-henryde01,RB,2018,NaT,NaN,NaN,NaN,NaN
4765,2016,2,45,Tennessee Titans,Derrick Henry,/players/derrick-henry-henryde01,RB,2019,NaT,NaN,NaN,NaN,NaN
4766,2016,2,45,Tennessee Titans,Derrick Henry,/players/derrick-henry-henryde01,RB,2020,NaT,NaN,NaN,NaN,NaN
4767,2016,2,45,Tennessee Titans,Derrick Henry,/players/derrick-henry-henryde01,RB,2021,2021-11-01,Tennessee (NFL),Placed on Injured Reserve (Foot),2021.0,11.0
4768,2016,2,45,Tennessee Titans,Derrick Henry,/players/derrick-henry-henryde01,RB,2022,NaT,NaN,NaN,NaN,NaN
4769,2016,2,45,Tennessee Titans,Derrick Henry,/players/derrick-henry-henryde01,RB,2023,NaT,NaN,NaN,NaN,NaN


Merged successfully! We see from the above that Derrick Henry's IR transaction matches up with the correct season (2021).

## 1.3.1 Season numbers, injuries and sequencing IR Data

Now that we have both the player's draft, seasons (ie calendar years) played and associated IR transactions combined, we'll create a sequence number representing how many years into their career they played. We'll assume that the first year they were drafted is Year 1 and every subsequent year they play will be incremented by 1.  This allows for comparability between players regardless of what actual years they played.




In [358]:
# Let's obtain the season number for the player - to make this easy I'm breaking out pieces of the transaction date into separate columns
# I'll assume that the NFL Season ends in February, and thus will attribute transactions that happen in Jan/Feb to the prior year (hence the -1)


p_ir['season_num'] = p_ir.apply(lambda x: x['season'] - x['year'] + 1,
                                axis=1)

p_ir[p_ir['player']=='Derrick Henry'][['season','season_num']]

,season,season_num
4762,2016,1
4763,2017,2
4764,2018,3
4765,2019,4
4766,2020,5
4767,2021,6
4768,2022,7
4769,2023,8


In [359]:
# Let's check on Zack Moss (who was placed on IR in January of '21) and verify whether the season attribution logic worked
p_ir[p_ir['player']=='Zack Moss']

,year,round,pick,draft_team,player,page,position,season,transdate,team,transaction,trans_year,trans_month,season_num
9921,2020,3,86,Buffalo Bills,Zack Moss,/players/zack-moss-mossza01,RB,2020,2021-01-12,Buffalo (NFL),Placed on Injured Reserve (Ankle),2021.0,1.0,1
9922,2020,3,86,Buffalo Bills,Zack Moss,/players/zack-moss-mossza01,RB,2021,NaT,NaN,NaN,NaN,NaN,2
9923,2020,3,86,Buffalo Bills,Zack Moss,/players/zack-moss-mossza01,RB,2022,NaT,NaN,NaN,NaN,NaN,3
9924,2020,3,86,Buffalo Bills,Zack Moss,/players/zack-moss-mossza01,RB,2023,NaT,NaN,NaN,NaN,NaN,4


Next we want to extract out the injury information from the transaction (ie what injury or body part was injured). We use Regex to sus out the actual string in the parentheses of each transaction. There are a number of transactions that are specified as Designated to Return in parantheses followed by the actual injury (also in parens) so our logic has to be robust to those cases

In [360]:
import re
import math

In [361]:
def extract_injury_from_transaction(x):
    
    """
    Extracts a string from a transaction using regex
    
    Parameters:
        x: str.  A string representing the actual transaction. Example: Placed on Injured Reserve (Ankle)
    Returns:
        str. A string representing the body part or injury which is assumed to be within parantheses. Returns "" if
        nothing found
    """
    
    if type(x) != float:
        for match in re.finditer(r"\([A-Za-z\s]+\)", x):
            if match.group() != "(Designated for Return)":
                try:
                    txt = re.sub(r"\(","",match.group())
                    txt = re.sub(r"\)","",txt)
                    return txt
                except:
                    print('error', match.group())
    else:
        return ""
        

In [362]:
# Let's extract the injury - we can use regex to extract the word(s) between the parentheses
p_ir['injury'] = p_ir['transaction'].apply(lambda x: extract_injury_from_transaction(x))

In [363]:
# p_ir.to_csv('data/all_ir_transactions.csv', index=False)
p_ir.to_csv('data/all_pos_ir_transactions.csv', index=False)

In [364]:
# Review the unique list of IR-related transactions
p_ir = pd.read_csv('data/all_pos_ir_transactions.csv')

In [365]:
p_ir[p_ir['player']=='Zack Moss']

,year,round,pick,draft_team,player,page,position,season,transdate,team,transaction,trans_year,trans_month,season_num,injury
9921,2020,3,86,Buffalo Bills,Zack Moss,/players/zack-moss-mossza01,RB,2020,2021-01-12,Buffalo (NFL),Placed on Injured Reserve (Ankle),2021.0,1.0,1,Ankle
9922,2020,3,86,Buffalo Bills,Zack Moss,/players/zack-moss-mossza01,RB,2021,NaN,NaN,NaN,NaN,NaN,2,NaN
9923,2020,3,86,Buffalo Bills,Zack Moss,/players/zack-moss-mossza01,RB,2022,NaN,NaN,NaN,NaN,NaN,3,NaN
9924,2020,3,86,Buffalo Bills,Zack Moss,/players/zack-moss-mossza01,RB,2023,NaN,NaN,NaN,NaN,NaN,4,NaN


In [366]:
# Quick eval that injury strings look correct and get counts
p_ir.injury.value_counts()

injury
Knee           650
Ankle          274
Hamstring      221
Shoulder       171
Undisclosed    167
Foot           153
Concussion      74
Groin           68
Achilles        67
Back            59
Pectoral        58
Leg             53
Calf            50
Neck            37
Hip             37
Toe             36
Quadriceps      36
Elbow           34
Biceps          31
Wrist           31
Hand            29
Abdomen         19
Ribs            19
Thumb           19
Thigh           11
Forearm         10
Triceps         10
Clavicle         9
Finger           9
Chest            8
Core Muscle      7
Hernia           7
Arm              6
Lisfranc         5
Oblique          4
Head             4
Kidney           4
Throat           2
Illness          2
Shin             2
Upper Body       2
Spine            1
Eye              1
Spleen           1
Heel             1
Jaw              1
Pelvis           1
Heart            1
Lower Body       1
Name: count, dtype: int64

In [367]:
p_ir = p_ir.set_index('player')

In [368]:
p_ir.head()

,year,round,pick,draft_team,page,position,season,transdate,team,transaction,trans_year,trans_month,season_num,injury
player,,,,,,,,,,,,,,
Eric Fisher,2013,1,1,Kansas City Chiefs,/players/eric-fisher-fisheer01,OT,2013,NaN,NaN,NaN,NaN,NaN,1,NaN
Eric Fisher,2013,1,1,Kansas City Chiefs,/players/eric-fisher-fisheer01,OT,2014,NaN,NaN,NaN,NaN,NaN,2,NaN
Eric Fisher,2013,1,1,Kansas City Chiefs,/players/eric-fisher-fisheer01,OT,2015,NaN,NaN,NaN,NaN,NaN,3,NaN
Eric Fisher,2013,1,1,Kansas City Chiefs,/players/eric-fisher-fisheer01,OT,2016,NaN,NaN,NaN,NaN,NaN,4,NaN
Eric Fisher,2013,1,1,Kansas City Chiefs,/players/eric-fisher-fisheer01,OT,2017,NaN,NaN,NaN,NaN,NaN,5,NaN


## 1.3.2 Creating the Career Frame ##

In [369]:
# Next we want to pivot the player's career so that the season numbers are columns with the injuries
# (if any) populating the season number they were incurred.

career = p_ir.groupby(['player','position','season_num'])['injury'].first().unstack()

In [370]:
career.sample(3)

,season_num,1,2,3,4,5,6,7,8,9,10,11
player,position,,,,,,,,,,,
Chris Paul,OG,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Anthony Averett,DB,None,None,Shoulder,None,Thumb,NaN,NaN,NaN,NaN,NaN,NaN
Jordan Morgan,OG,Undisclosed,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


An interesting thing happened...all of the seasons where an injury transaction is not captured, we see None, for any season that wasn't played by the player, we see NaN.  This makes a nice division between seasons that the player was healthy (designated as None) and the end point in the sequence of their career (their last season) as NaN.  Though this wasn't intentional, it's certainly appreciated.

I'm going to update each series (column) with Healthy if it spots a "None" value

In [341]:
for i in range(1,12):
    career[i] = career[i].apply(lambda x: "Healthy" if x== None else x)

In [342]:
career.sample(4)

,season_num,1,2,3,4,5,6,7,8,9,10,11
player,position,,,,,,,,,,,
Pita Taumoepenu,LB,Healthy,Healthy,Hamstring,Healthy,Healthy,NaN,NaN,NaN,NaN,NaN,NaN
Nicholas Petit-Frere,OT,Healthy,Shoulder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Talanoa Hufanga,DB,Healthy,Healthy,Knee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cameron Latu,TE,Knee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [372]:
# Save work.
# career.to_csv('data/ir_career.csv') # for just the skill positions drafted
career.to_csv('data/all_pos_ir_career.csv') # for all positions

## 1.4.0 Injury Lengths by IR Type

In addition to representing the career as a sequence of healthy/injuries, we will also perform some analysis on injury duration. More specifically, we want to know how many games a player may miss based on the type of injury incurred.  The transaction log above provided insight into when a player was placed on IR as well as the injury. However, it didn't provide details of what games were played during that time.  We could infer it by counting the weeks...however, the data to be extracted provides additional details such as games missed outside of IR.

In [152]:
trans = pd.read_csv('data/all_pos_full_transactions.csv')

In [160]:
ir = trans[trans['transaction'].str.contains('Injured Reserve')]

In [162]:
ir.transaction.value_counts().index

Index(['Placed on Injured Reserve (Knee)', 'Activated from Injured Reserve',
       'Designated for return from Injured Reserve',
       'Placed on Injured Reserve (Ankle)', 'Waived from Injured Reserve',
       'Placed on Injured Reserve (Undisclosed)',
       'Placed on Injured Reserve (Hamstring)',
       'Placed on Injured Reserve (Shoulder)', 'Placed on Injured Reserve',
       'Placed on Injured Reserve (Foot)', 'Released from Injured Reserve',
       'Placed on Injured Reserve (Achilles)',
       'Placed on Injured Reserve (Concussion)',
       'Placed on Injured Reserve (Groin)', 'Placed on Injured Reserve (Back)',
       'Placed on Injured Reserve (Pectoral)',
       'Placed on Injured Reserve (Leg)', 'Placed on Injured Reserve (Calf)',
       'Placed on Injured Reserve (Quadriceps)',
       'Placed on Injured Reserve (Neck)', 'Placed on Injured Reserve (Hip)',
       'Placed on Injured Reserve (Toe)', 'Placed on Injured Reserve (Elbow)',
       'Placed on Injured Reserve (Bic

In [ ]:
files = [pd.read_csv(f'{x}_transactions.csv', parse_dates=['transaction_date']) for x in years]

In [165]:
df.head()

,year,round,pick,draft_team,player,page,position,season
0,2013,1,1,Kansas City Chiefs,Eric Fisher,/players/eric-fisher-fisheer01,OT,2013
0,2013,1,1,Kansas City Chiefs,Eric Fisher,/players/eric-fisher-fisheer01,OT,2014
0,2013,1,1,Kansas City Chiefs,Eric Fisher,/players/eric-fisher-fisheer01,OT,2015
0,2013,1,1,Kansas City Chiefs,Eric Fisher,/players/eric-fisher-fisheer01,OT,2016
0,2013,1,1,Kansas City Chiefs,Eric Fisher,/players/eric-fisher-fisheer01,OT,2017


## 1.4.1 Extract Game Log

The following scrapes each player's gamelog from footballdb.com where each row in a table represents a game.  If the player didn't play that game, the row will have a colspan (ie merges several cells in the table)  and the text in that colspan will indicate why the player missed the game. We can use that text to determine IR-attribution.

The resulting data object will look like:

<code>{"Player A": {"2022": [{Game 1 Object}, {Game 2 Object}], "2023": [{Game 1 Object}...]}</code>

where each player page is a key, and the value is a nested dictionary of seasons that the player played with each season having a list of games dictionaries.

In [69]:
mydict = {}
gamedates = []
team = []
opp =[]
played = []

head = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15"}


for page in tqdm(pages):
    if page not in mydict.keys():
        time.sleep(2)
        seasons = df[df['page'] == page]['season'].values
        mydict[page] = {}
        mydict[page]['season'] = []

        
        seasons = seasons[0]
        for season in seasons:

            gamedates = []
            team = []
            opp =[]
            played = []
            gamelog = []
            url = f"https://www.footballdb.com/{page}/gamelogs/{season}"
            soup = getSoup(url,head)
            try:
                table =  soup.find_all('table')[0]
            except:
                print('error')
                continue
            for row in table.find_all('tr'):
                classname = row['class']
            
                a = 'header' not in classname
                b = 'preseason' not in classname
                # c = 'NFL' in row
                if a and b:
                    cells = row.find_all('td')
                    played = 0 if cells[4].has_attr('colspan') else 1
                    if played == 0:
                        # if the player didn't play, look at the descendant tag's string to get why
                        # options include ('Injured Reserve', 'Inactive', etc.)
                        reason = cells[4].contents[0].get_text()
                    else:
                        reason = ''
                    game_data = {'date': cells[0].get_text(),
                                 'team': cells[1].get_text(),
                                 'opp': cells[2].get_text(),
                                 'played': played,
                                 'reason': reason
                                 
                                }
                    
                    gamelog.append(game_data)
            mydict[page]['season'].append({season:gamelog})
    


 32%|███▏      | 896/2817 [1:05:15<2:05:25,  3.92s/it]

error


 41%|████      | 1146/2817 [1:22:35<1:32:30,  3.32s/it]

error


100%|██████████| 2817/2817 [2:53:21<00:00,  3.69s/it]  


In [71]:
# mydict

In [72]:
import json

In [73]:
json.dump(mydict, open('gamelog.json','w'))

In [266]:
mydict = json.load(open('gamelog.json','r'))

## 1.4.2 Create GameLog DataFrame ##

In [74]:
def create_gamelog_frame(mydict):

    """
    Produces a dataframe of player games for each season in the players career with a flag to indicate 
    whether they played the corresponding game.
    
    Parameters:
        mydict (dictionary): a dictionary of player seasons and associated game logs for the season.
    Returns:
        dataframe in the form of player|season|game|played flag
    """

    dates = []
    season_lst = []
    players = []
    played = []
    reasons = []
    
    for k,v in mydict.items():
        for _, seasons in v.items():
            for season in seasons:
                for year, val in season.items():
                    for game in val:
                        players.append(k)
                        season_lst.append(year)
                        dates.append(game['date'])
                        played.append(game['played'])
                        reasons.append(game['reason'])
    gamelog = pd.DataFrame({'player': players, 
              'season': season_lst,
              'date': dates,
              'played': played,
              'reason': reasons})
    
    gamelog['date'] =  pd.to_datetime(gamelog['date'])
    return gamelog
                    
                    

In [75]:
log = create_gamelog_frame(mydict)

/var/folders/jk/7_6237px1mz6nps611m4052w0000gn/T/ipykernel_11600/476056643.py:35: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  gamelog['date'] =  pd.to_datetime(gamelog['date'])


In [76]:
log

,player,season,date,played,reason
0,/players/eric-fisher-fisheer01,2013,2013-09-08,1,
1,/players/eric-fisher-fisheer01,2013,2013-09-15,1,
2,/players/eric-fisher-fisheer01,2013,2013-09-19,1,
3,/players/eric-fisher-fisheer01,2013,2013-09-29,1,
4,/players/eric-fisher-fisheer01,2013,2013-10-06,0,Inactive
...,...,...,...,...,...
172474,/players/desjuan-johnson-johnsde24,2023,2023-12-17,0,Inactive
172475,/players/desjuan-johnson-johnsde24,2023,2023-12-21,0,Inactive
172476,/players/desjuan-johnson-johnsde24,2023,2023-12-31,1,
172477,/players/desjuan-johnson-johnsde24,2023,2024-01-07,1,


In [77]:
log.to_csv('data/gamelog.csv', index=False)

<font size=4>Now that the data has been extracted and processed, we'll move to Notebook 2 where explore the data in a bit more depth.